In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
from torch import nn
from torch.utils.data import TensorDataset,Dataset,DataLoader,random_split
import datetime
import pandas as pd

# prepare data

In [2]:
with open('config.npy', 'rb') as f:
    config = np.load(f)
with open('J.npy', 'rb') as f:
    J = np.load(f).reshape(1000,1,32)
with open('energy.npy', 'rb') as f:
    # energy 都是负的，在这里取 正值
    energy = -np.load(f)

In [3]:
J.shape

(1000, 1, 32)

In [4]:
data = TensorDataset(torch.tensor(J).float(),torch.tensor(energy).reshape(len(energy),1).float())

n_train = int(len(data)*0.8)
n_valid = len(data) - n_train
ds_train,ds_valid = random_split(data,[n_train,n_valid])

dl_train,dl_valid = DataLoader(ds_train,batch_size = 8),DataLoader(ds_valid,batch_size = 8)


In [5]:
for features,labels in dl_train:
    print(features.shape)
    print(labels.shape)
    break

torch.Size([8, 1, 32])
torch.Size([8, 1])


# prepare model

In [6]:
class Net(nn.Module): 
    def __init__(self):
        super(Net, self).__init__()
        #32
        self.conv1d1 = nn.Conv1d(in_channels=1,out_channels=1,kernel_size=3)
        self.maxpool1 = nn.MaxPool1d(3,stride=1)
        #28
        
        self.conv1d2 = nn.Conv1d(in_channels=1,out_channels=1,kernel_size=3)
        self.maxpool2 = nn.MaxPool1d(3,stride=1)
        #24
        
        self.conv1d3 = nn.Conv1d(in_channels=1,out_channels=1,kernel_size=3)
        self.maxpool3 = nn.MaxPool1d(3,stride=1)
        #20
        
        self.conv1d4 = nn.Conv1d(in_channels=1,out_channels=1,kernel_size=3)
        self.maxpool4 = nn.MaxPool1d(3,stride=1)
        #16
        
        self.linear1 = nn.Linear(28,1)
        self.lrelu1 = nn.LeakyReLU()
#         self.linear2 = nn.Linear(8,4)
#         self.lrelu2 = nn.LeakyReLU()
#         self.linear3 = nn.Linear(4,1)
#         self.lrelu3 = nn.LeakyReLU()
        
    def forward(self,x):
        x = self.conv1d1(x)
        x = self.maxpool1(x)
#         x = self.conv1d2(x)
#         x = self.maxpool2(x)
#         x = self.conv1d3(x)
#         x = self.maxpool3(x)
#         x = self.conv1d4(x)
#         x = self.maxpool4(x)
        
        x = self.linear1(x)
        y = self.lrelu1(x)
        
#         x = self.linear2(x)
#         x = self.lrelu2(x)
        
#         x = self.linear3(x)
#         y = self.lrelu3(x)
#         y = y.reshape(len(y),1)
        return y
        
model = Net()

In [1]:
c = nn.Conv1d(in_channels=1,out_channels=1,kernel_size=3)
m = nn.MaxPool1d(3, stride=1)
input = torch.randn(1000,1,32)
# output = m(c(input))
output = model(input)
output.shape

NameError: name 'nn' is not defined

In [ ]:
# from torchkeras import summary
# summary(model,input_shape= (1000,32))

In [ ]:
model.optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)
model.loss_func = torch.nn.MSELoss()
model.metric_func = lambda y_pred,y_true: torch.mean(1-torch.abs(y_true-y_pred))
model.metric_name = "auc"

In [ ]:
def train_step(model,features,labels):
    
    # 训练模式，dropout层发生作用
    model.train()
    
    # 梯度清零
    model.optimizer.zero_grad()
    
    # 正向传播求损失
    predictions = model(features)
    loss = model.loss_func(predictions,labels)
    metric = model.metric_func(predictions,labels)

    # 反向传播求梯度
    loss.backward()
    model.optimizer.step()

    return loss.item(),metric.item()

def valid_step(model,features,labels):
    
    # 预测模式，dropout层不发生作用
    model.eval()
    # 关闭梯度计算
    with torch.no_grad():
        predictions = model(features)
        loss = model.loss_func(predictions,labels)
        metric = model.metric_func(predictions,labels)
    
    return loss.item(), metric.item()


# 测试train_step效果
features,labels = next(iter(dl_train))
labels = labels.reshape(8,1)
train_step(model,features,labels)

In [ ]:
def train_model(model,epochs,dl_train,dl_valid,log_step_freq):

    metric_name = model.metric_name
    print("Start Training...")
    nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("=========="*8 + "%s"%nowtime)
    df = pd.DataFrame([], columns=['EPOCH','loss','auc','val_loss','val_auc'])
    for epoch in range(1,epochs+1):  

        # 1，训练循环-------------------------------------------------
        loss_sum = 0.0
        metric_sum = 0.0
        step = 1

        for step, (features,labels) in enumerate(dl_train, 1):

            loss,metric = train_step(model,features,labels)

            # 打印batch级别日志
            loss_sum += loss
            metric_sum += metric
            if step%log_step_freq == 0:   
                print(("[step = %d] loss: %.3f, "+metric_name+": %.3f") %
                      (step, loss_sum/step, metric_sum/step))

        # 2，验证循环-------------------------------------------------
        val_loss_sum = 0.0
        val_metric_sum = 0.0
        val_step = 1

        for val_step, (features,labels) in enumerate(dl_valid, 1):

            val_loss,val_metric = valid_step(model,features,labels)

            val_loss_sum += val_loss
            val_metric_sum += val_metric

        # 3，记录日志-------------------------------------------------
        info = (epoch, loss_sum/step, metric_sum/step, 
                val_loss_sum/val_step, val_metric_sum/val_step)
        df = df.append(pd.DataFrame([info], columns=['EPOCH','loss','auc','val_loss','val_auc']),ignore_index=True)
        # 打印epoch级别日志
        print(("\nEPOCH = %d, loss = %.3f,"+ metric_name + \
              "  = %.3f, val_loss = %.3f, "+"val_"+ metric_name+" = %.3f") 
              %info)
        nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print("\n"+"=========="*8 + "%s"%nowtime)

    print('Finished Training...')
    
    return df

In [ ]:
epochs = 10

history = train_model(model,epochs,dl_train,dl_valid,log_step_freq = 50)

In [ ]:
history

In [ ]:
# 测试model效果
features_test,labels_test = next(iter(dl_valid))
# labels = labels.reshape(8,1)
predictions_test = model(features_test)

In [ ]:
predictions_test.shape

In [ ]:
labels_test